#Create CNN Model and Optimize it using Keras Tuner

##Preprocessing

In [ ]:
!pip install keras-tuner

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.8.2'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() 

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

##Building a Model

In [9]:
from tensorflow.python import metrics
def build_model(hp):                                                                                    #hp is a abbr for hyper-parameter
  model = keras.Sequential([
      keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),        #number of filters will be chosen between 64 and 128 with a step of 16
                          kernel_size=hp.Choice('conv_1_kernel', values=[3,5,7]),                       #kernel size will be chosen from the given values
                          activation=keras.activations.relu,                                            #ReLU activation
                          input_shape=(28,28,1)),

      keras.layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                          kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                          activation=keras.activations.relu),

      keras.layers.Flatten(),                                                                           #Flatten the input for the dense layers

      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),                          #number of dense units will be chosen between 32 and 128.
          activation=keras.activations.relu),

      keras.layers.Dense(units=10, activation=keras.activations.softmax)
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), #Adam optimizer with varying learning rate
                loss=keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model

In [11]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters


In [12]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, 
                            directory='output', project_name="Mnist_Fashion")             #random search using the build model function

In [13]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)           #search for the best model

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.8514999747276306

Best val_accuracy So Far: 0.9079999923706055
Total elapsed time: 00h 02m 42s


In [14]:
best_model = tuner_search.get_best_models(num_models=1)[0]                #pick the best model, can pick multiple best models if required

In [15]:
best_model.summary()                                     

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 112)       268912    
                                                                 
 flatten (Flatten)           (None, 54208)             0         
                                                                 
 dense (Dense)               (None, 48)                2602032   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 2,872,394
Trainable params: 2,872,394
Non-trainable params: 0
_________________________________________________________________


In [16]:
best_model.fit(train_images, train_labels, epochs=10, 
               validation_split=0.1, initial_epoch=3)          #initial_epoch=3 means the training starts at epoch 4 as we have already trained for 3 epochs in the model selection 

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1424 - accuracy: 0.9462 - val_loss: 0.2762 - val_accuracy: 0.9070
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1021 - accuracy: 0.9615 - val_loss: 0.2792 - val_accuracy: 0.9135
Epoch 6/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0752 - accuracy: 0.9715 - val_loss: 0.3618 - val_accuracy: 0.9107
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0527 - accuracy: 0.9807 - val_loss: 0.3742 - val_accuracy: 0.9098
Epoch 8/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0417 - accuracy: 0.9844 - val_loss: 0.4119 - val_accuracy: 0.9095
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0365 - accuracy: 0.9871 - val_loss: 0.4930 - val_accuracy: 0.9027
Epoch 10/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0271 - accuracy: 0.9903 - val_loss: 0.5263 - val_a

##Check Accuracy

In [23]:
from sklearn.metrics import accuracy_score
y_pred_prob = best_model.predict(test_images)         #model.predict returns an array of probabilities for each image 

In [24]:
y_pred = np.argmax(y_pred_prob, axis=1)               #pick the maximum probability as the prediction
y_pred

array([9, 2, 1, ..., 8, 1, 5])

In [25]:
accuracy_score(y_pred, test_labels)

0.9063